In [25]:
import numpy as np
import pandas as pd

In [26]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(40000)
user_df = pd.read_csv('Users.csv')

<ipython-input-26-e957fbe9ed49>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [27]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [28]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40000 entries, 719620 to 898627
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      40000 non-null  int64 
 1   ISBN         40000 non-null  object
 2   Book-Rating  40000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [29]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [30]:
ratings_df.isnull().sum()


,0
User-ID,0
ISBN,0
Book-Rating,0


In [31]:
book_df.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


In [32]:
book_df.dropna(how='any', inplace=True)

In [33]:
user_df.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [34]:
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')